In [25]:
# import libraries
import os
import re
import pathlib
from glob import glob

import matplotlib.pyplot as plt
import earthaccess
import xrspatial
import geopandas as gpd
import rioxarray as rxr
import rioxarray.merge as rxrmerge

In [26]:
# load stored variables
%store -r shey_gdf shey_gdf c_soil_url_list p_soil_url_list

In [27]:
# build project and elevation directories

data_dir = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data'
)
project_dir = os.path.join(data_dir, 'habitat_suitability')
elevation_dir = os.path.join(data_dir, 'srtm')

os.makedirs(elevation_dir, exist_ok=True)

In [28]:
# login to earthaccess
earthaccess.login(strategy="interactive", persist=True)

In [29]:
# search for the appropriate DEM

datasets = earthaccess.search_datasets(keyword='SRTM DEM', count=11)
for dataset in datasets:
    print(dataset['umm']['ShortName'], dataset['umm']['EntryTitle'])

NASADEM_SHHP NASADEM SRTM-only Height and Height Precision Mosaic Global 1 arc second V001
NASADEM_SIM NASADEM SRTM Image Mosaic Global 1 arc second V001
NASADEM_SSP NASADEM SRTM Subswath Global 1 arc second V001
C_Pools_Fluxes_CONUS_1837 CMS: Terrestrial Carbon Stocks, Emissions, and Fluxes for Conterminous US, 2001-2016
SRTMGL1 NASA Shuttle Radar Topography Mission Global 1 arc second V003
GEDI01_B GEDI L1B Geolocated Waveform Data Global Footprint Level V002
GEDI02_B GEDI L2B Canopy Cover and Vertical Profile Metrics Data Global Footprint Level V002
NASADEM_HGT NASADEM Merged DEM Global 1 arc second V001
SRTMGL3 NASA Shuttle Radar Topography Mission Global 3 arc second V003
SRTMGL1_NC NASA Shuttle Radar Topography Mission Global 1 arc second NetCDF V003
SRTMGL30 NASA Shuttle Radar Topography Mission Global 30 arc second V002


In [30]:
# shey_gdf.bounds
# bounds_buffer= (shey_gdf.total_bounds + 0.25)

In [31]:
# srtm_c_results = earthaccess.search_data(
#     short_name="SRTMGL1",
#     bounding_box=bounds_buffer    
# )
# print(srtm_c_results)  # See if any results are found


In [32]:
## Creating a pattern for selecting srtm tiles based on the grassland gdf bounds

# Extract bounding box from shey_gdf
xmin, ymin, xmax, ymax = shey_gdf.total_bounds

# Generate SRTM tile names based on integer degree tiles
latitudes = range(int(ymin), int(ymax) + 1)
longitudes = range(int(xmin), int(xmax) + 1)

# Create SRTM file patterns
srtm_c_pattern_list = []
for lat in latitudes:
    for lon in longitudes:
        lat_prefix = f"N{lat}" if lat >= 0 else f"S{abs(lat)}"
        lon_prefix = f"E{lon}" if lon >= 0 else f"W{abs(lon)}"
        srtm_c_pattern_list.append(os.path.join(elevation_dir, f"{lat_prefix}{lon_prefix}.hgt.zip"))

# # Use glob to find matching files
# srtm_c_pattern = [glob(pattern) for pattern in srtm_c_pattern_list]
# srtm_c_pattern = [item for sublist in srtm_c_pattern for item in sublist]  # Flatten list

# print("Matching SRTM files:", srtm_c_pattern)


In [33]:
srtm_c_pattern_list

['C:\\Users\\moenc\\earth-analytics\\data\\srtm\\N46W97.hgt.zip',
 'C:\\Users\\moenc\\earth-analytics\\data\\srtm\\N46W96.hgt.zip']

In [34]:
print(os.listdir(elevation_dir))

['N36W103.SRTMGL1.hgt.zip', 'N36W104.SRTMGL1.hgt.zip', 'N36W105.SRTMGL1.hgt.zip', 'N37W103.SRTMGL1.hgt.zip', 'N37W104.SRTMGL1.hgt.zip', 'N37W105.SRTMGL1.hgt.zip', 'N40W104.SRTMGL1.hgt.zip', 'N40W105.SRTMGL1.hgt.zip', 'N41W104.SRTMGL1.hgt.zip', 'N41W105.SRTMGL1.hgt.zip', 'N43W102.SRTMGL1.hgt.zip', 'N43W103.SRTMGL1.hgt.zip', 'N43W104.SRTMGL1.hgt.zip', 'N43W105.SRTMGL1.hgt.zip', 'N46W103.SRTMGL1.hgt.zip', 'N46W104.SRTMGL1.hgt.zip', 'N46W105.SRTMGL1.hgt.zip', 'N47W103.SRTMGL1.hgt.zip', 'N47W104.SRTMGL1.hgt.zip', 'N47W105.SRTMGL1.hgt.zip', 'N48W103.SRTMGL1.hgt.zip', 'N48W104.SRTMGL1.hgt.zip', 'N48W105.SRTMGL1.hgt.zip']


In [35]:
# Define a pattern to identify DEM tiles associated with shey National Grassland.
srtm_c_pattern = [
    os.path.join(elevation_dir, 'N43*hgt.zip'),
]
bounds_c = tuple(shey_gdf.total_bounds)
buffer = 0.25
xmin, ymin, xmax, ymax = bounds_c
bounds_buffer = (xmin-buffer, ymin-buffer, xmax+buffer, ymax+buffer)

# Fix the first code block.
all_files = []
for pattern in srtm_c_pattern:
    all_files.extend(glob(pattern))

if not all_files:
    srtm_c_results = earthaccess.search_data(
        short_name="SRTMGL1",
        bounding_box=bounds_c
    )
    srtm_c_results = earthaccess.download(srtm_c_results, elevation_dir)

# if not glob(srtm_c_pattern):
#     srtm_c_results = earthaccess.search_data(
#         short_name = "SRTMGL1",
#         bounding_box=bounds_buffer    
#     )
#     srtm_c_results = earthaccess.download(srtm_c_results, elevation_dir)

In [ ]:
import zipfile

# Path to the downloaded ZIP file
zip_path = strm_

# Folder where you want to extract the contents
extract_path = 'extracted_folder'

# Open the ZIP file in read mode
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # Extract all contents to the specified folder
    zip_ref.extractall(extract_path)

print("Unzipping complete")

In [36]:
# # Print the DEM for shey natl. Grassland
# srtm_c_da_list=[]
# for srtm_c_path in glob(srtm_c_pattern):
#     tile_da = rxr.open_rasterio(srtm_c_path, mask_and_scale=True).squeeze()
#     cropped_da = tile_da.rio.clip_box(*bounds_buffer)
#     srtm_c_da_list.append(cropped_da)
    
# srtm_c_da = rxrmerge.merge_arrays(srtm_c_da_list)
# srtm_c_da.plot(cmap='terrain')
# shey_gdf.boundary.plot(ax=plt.gca(), color='black')

# Fix the second code block.
srtm_c_da_list = []
for pattern in srtm_c_pattern:
    for srtm_c_path in glob(pattern):
        tile_da = rxr.open_rasterio(srtm_c_path, mask_and_scale=True).squeeze()
        cropped_da = tile_da.rio.clip_box(*bounds_buffer)
        srtm_c_da_list.append(cropped_da)

srtm_c_da = rxrmerge.merge_arrays(srtm_c_da_list)
srtm_c_da.plot(cmap='terrain')
shey_gdf.boundary.plot(ax=plt.gca(), color='white')

NoDataInBounds: No data found in bounds.